In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import time


In [2]:
# Convert series to supervised learning function (unchanged)
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.fillna(0, inplace=True)
    return agg

In [3]:
# Load and preprocess the data
file_path = "/home/shreevidyag/workspace/Data_bengaluru/combined_cleaned.csv"
data = pd.read_csv(file_path)
data.head()
data.loc[:, 'PM25'] = data['PM25'].fillna(0)
data = data[24:]
data.to_csv('pollution1.csv')
dataset = pd.read_csv('pollution1.csv', header=0, index_col=0)
values = dataset.values
values = values.astype('float32')

In [4]:
# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(values[:, :-1])
scaled_label = scaler.fit_transform(values[:, -1].reshape(-1, 1))
values = np.column_stack((scaled_features, scaled_label))


In [5]:
# Convert to supervised learning format
reframed = series_to_supervised(values, 1, 1, True)
reframed.drop(reframed.columns[7:12], axis=1, inplace=True)

In [6]:
# Split data into training and test sets
n_train_hours = 120000
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [7]:
# Reshape data into 3D for LSTM (samples, timesteps, features)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_X = torch.tensor(train_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)


In [8]:
# Define the LSTM model with attention mechanism in PyTorch
class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMWithAttention, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.attn = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_out, _ = self.attn(lstm_out, lstm_out, lstm_out)
        output = self.fc(attn_out[:, -1, :])  # take the last time step output
        return output

In [9]:
# Initialize model, loss function, and optimizer
model = LSTMWithAttention(input_size=train_X.shape[2], hidden_size=128, output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
start = time.time()
num_epochs = 1000
batch_size = 72

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    output = model(train_X)
    loss = criterion(output, train_y.view(-1, 1))
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item()}")


Epoch 0/1000, Loss: 0.2533097267150879
Epoch 5/1000, Loss: 0.1141921728849411
Epoch 10/1000, Loss: 0.05391766503453255
Epoch 15/1000, Loss: 0.055849339812994


In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    yhat = model(test_X).numpy()

In [ ]:
# Inverse scaling
inv_yhat = np.concatenate((yhat, test_X[:, 1:, :].numpy()[:, :, 0]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:, 0]

test_y = test_y.numpy().reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:, :].numpy()[:, :, 0]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, 0]


In [ ]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print(f'Test RMSE: {rmse:.3f}')

In [ ]:
# Plot results
def plot_predicted(predicted_data, true_data):
    plt.figure(figsize=(17, 8))
    plt.plot(true_data, label='True Data', color='green', linewidth=3)
    plt.plot(predicted_data, label='Prediction', color='red', linewidth=2)
    plt.legend()
    plt.show()

plot_predicted(inv_yhat[:300], inv_y[:300])